### RAG implemenation with Symantic Chunking & OpenSearch as Vector Store (with hsnw index)

In [ ]:
# Load the enviornment variables
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [ ]:
## Import Open AI embedding model used for symantic chunking & while convetring the chunks in vectors
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# Load the pdf document and create chunks using symantic chunking 
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

FILE_PATH=r"C:\Users\gauravkkumar\Documents\2025\rag\data\2407_01502v1.pdf"
loader=PyPDFLoader(FILE_PATH)
pages = loader.load()

text_splitter = SemanticChunker(embeddings)
split_docs = text_splitter.split_documents(pages)

print(split_docs[0].page_content)

### OpenSearch 
1. Create the client
2. Create the index
3. Create the OpenSearch vector store 
Pre-requiste: Make sure Opensearch is running in Docker locally

In [ ]:
## Create OpenSearch Clinet 
from langchain_community.vectorstores import OpenSearchVectorSearch
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', '<password>'),  # Replace with your credentials
    use_ssl=True,
    verify_certs=False
)

In [ ]:
len(embeddings.embed_query("Hello"))

In [ ]:
## Create the index

index_name = "langchain-demo-index"

index_body = {
    "settings": {
        "index": {
            "knn": True
        }
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 3072,  # Set to your embedding dimension
                "method": {
                    "engine": "faiss",  # or "nmslib" if desired
                    "space_type": "l2",  # or "cosinesimil", etc.
                    "name": "hnsw",      # or "ivf", "flat", etc. (see docs)
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    }
                }
            }
        }
    }
}

if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name, body=index_body)

In [ ]:
docsearch = OpenSearchVectorSearch(embedding_function=embeddings, 
                                   index_name=index_name,
                                   opensearch_url="https://localhost:9200", 
                                    http_auth=("admin", "<password>"),
                                    use_ssl = True,
                                    verify_certs = False, 
)

In [ ]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(split_docs))]

In [ ]:
docsearch.add_documents(documents=split_docs, ids=uuids)

In [ ]:
count = client.count(index=index_name)['count']
count

In [ ]:
results = docsearch.similarity_search("What is agentic AI?")

In [ ]:
results

In [ ]:
retriever=docsearch.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.3} #hyperparameter
)
retriever.invoke("What is LangChain")

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("what is Agentic AI?")